In [ ]:
import geopandas as gpd
import pandas as pd
import os

In [ ]:
RUTA_TIENDAS_CON_CENSO_DENUE_OSM_INPUT = './datos/output/tiendas_train_censo_denue_osm.gpkg'
LAYER_TIENDAS_INPUT = 'tiendas_censo_denue_osm' 
RUTA_IMU_CSV_DIR = './datos/imu_data/' 
ARCHIVO_IMU_AGEB_CSV = RUTA_IMU_CSV_DIR + 'indice_marginacion_urbana_ageb_2020.csv' # Tu archivo
RUTA_DATOS_OUTPUT = './datos/output/'
ARCHIVO_SALIDA_GPKG = RUTA_DATOS_OUTPUT + 'tiendas_train_completo_con_imu.gpkg'
COL_IMU_CVEGEO_EXISTENTE = 'CVE_AGEB'
COL_IMU_INDICE = 'IM_2020'    
COL_IMU_GRADO = 'GM_2020'    
COL_IMU_INDICE_NACIONAL = 'IMN_2020'

# Crear directorio de output si no existe
if not os.path.exists(RUTA_DATOS_OUTPUT):
    os.makedirs(RUTA_DATOS_OUTPUT)
if not os.path.exists(RUTA_IMU_CSV_DIR): # Solo por si acaso para el ejemplo
    os.makedirs(RUTA_IMU_CSV_DIR)
    print(f"Directorio creado: {RUTA_IMU_CSV_DIR}. Por favor, coloca tu archivo IMU allí.")
# --- Fin de Configuración ---

print("--- Iniciando Integración del Índice de Marginación Urbana (IMU) ---")

In [ ]:
# --- 1. Cargar el GeoDataFrame de Tiendas ---
try:
    tiendas_gdf = gpd.read_file(RUTA_TIENDAS_CON_CENSO_DENUE_OSM_INPUT, layer=LAYER_TIENDAS_INPUT)
    print(f"Tiendas cargadas: {len(tiendas_gdf)}. CRS: {tiendas_gdf.crs}")
    if 'CVEGEO' not in tiendas_gdf.columns: # La clave en tiendas_gdf se llama 'CVEGEO' (del shapefile)
        raise ValueError("Columna 'CVEGEO' no encontrada en tiendas_gdf. Se necesita para el merge con IMU.")
    tiendas_gdf['CVEGEO'] = tiendas_gdf['CVEGEO'].astype(str).str.strip()
except Exception as e:
    print(f"Error al cargar tiendas: {e}")
    raise

In [ ]:
# --- 2. Cargar y Procesar Datos del IMU ---
try:
    print(f"\nCargando datos del IMU desde: {ARCHIVO_IMU_AGEB_CSV}")
    imu_df = pd.read_csv(ARCHIVO_IMU_AGEB_CSV, encoding='utf-8-sig') 
    print(f"  {len(imu_df)} registros IMU cargados.")
    print("  Primeras filas del IMU:")
    print(imu_df.head())
    print("\n  Columnas disponibles en el IMU:")
    print(imu_df.columns.tolist())

    # Verificar columnas necesarias del IMU
    imu_cols_necesarias_para_analisis = [COL_IMU_CVEGEO_EXISTENTE, COL_IMU_INDICE, COL_IMU_GRADO, COL_IMU_INDICE_NACIONAL]
    missing_cols_imu = [col for col in imu_cols_necesarias_para_analisis if col not in imu_df.columns]
    if missing_cols_imu:
        raise ValueError(f"Columnas IMU para análisis faltantes: {missing_cols_imu}. Columnas disponibles: {imu_df.columns.tolist()}. Ajusta la configuración.")

    # Preparar la clave CVEGEO del IMU (a string, sin espacios)
    imu_df[COL_IMU_CVEGEO_EXISTENTE] = imu_df[COL_IMU_CVEGEO_EXISTENTE].astype(str).str.strip()
    print(f"  Clave '{COL_IMU_CVEGEO_EXISTENTE}' del IMU preparada como string.")
    
    # Seleccionar solo las columnas que necesitamos para el merge y análisis
    cols_imu_finales = [COL_IMU_CVEGEO_EXISTENTE, COL_IMU_INDICE, COL_IMU_GRADO, COL_IMU_INDICE_NACIONAL]
    imu_para_merge_df = imu_df[cols_imu_finales].drop_duplicates(subset=[COL_IMU_CVEGEO_EXISTENTE])

except FileNotFoundError:
    print(f"Error: Archivo IMU no encontrado en '{ARCHIVO_IMU_AGEB_CSV}'.")
    raise
except Exception as e:
    print(f"Error al cargar o procesar el archivo IMU: {e}")
    raise

In [ ]:
# --- 3. Unir (Merge) Datos del IMU con el GeoDataFrame de Tiendas ---
print(f"\nRealizando merge entre tiendas_gdf y datos del IMU usando 'CVEGEO' (de tiendas) y '{COL_IMU_CVEGEO_EXISTENTE}' (de IMU)...")
try:
    tiendas_final_gdf = tiendas_gdf.merge(
        imu_para_merge_df,
        left_on='CVEGEO',  # Clave en tiendas_gdf (viene del shapefile 00a.shp)
        right_on=COL_IMU_CVEGEO_EXISTENTE, # Clave en imu_df
        how='left'
    )
    print(f"Merge con IMU completado. Dataset resultante: {len(tiendas_final_gdf)} filas.")

    tiendas_sin_imu_match = tiendas_final_gdf[COL_IMU_INDICE].isnull().sum() 
    print(f"Número de tiendas sin datos del IMU correspondientes: {tiendas_sin_imu_match} de {len(tiendas_final_gdf)}")

    # Si la columna clave del IMU se añadió y es diferente de 'CVEGEO', y es redundante, eliminarla
    if COL_IMU_CVEGEO_EXISTENTE != 'CVEGEO' and COL_IMU_CVEGEO_EXISTENTE in tiendas_final_gdf.columns:
        tiendas_final_gdf = tiendas_final_gdf.drop(columns=[COL_IMU_CVEGEO_EXISTENTE])
        print(f"Columna clave redundante del IMU '{COL_IMU_CVEGEO_EXISTENTE}' eliminada.")
except Exception as e:
    print(f"Error durante el merge con IMU: {e}")
    print("Verifica que las columnas CVEGEO en ambos DataFrames tengan el mismo formato y valores coincidentes.")
    raise

In [ ]:
# --- 4. Guardar el Resultado ---
print("\n--- Guardando GeoDataFrame Enriquecido con IMU ---")
print("Primeras filas del dataset final con datos IMU:")
pd.set_option('display.max_columns', None)
print(tiendas_final_gdf.head())

print("\nColumnas finales:")
print(tiendas_final_gdf.columns.tolist())

try:
    tiendas_final_gdf.to_file(ARCHIVO_SALIDA_GPKG, driver='GPKG', layer='tiendas_completo_con_imu')
    print(f"\nDataset enriquecido guardado como GeoPackage en '{ARCHIVO_SALIDA_GPKG}'")
except Exception as e:
    print(f"Error al guardar el archivo de salida GeoPackage: {e}")

print("\n--- Integración del Índice de Marginación Urbana Completada ---")